In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ape = pd.read_csv("app_events.csv")

In [3]:
ape.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


# read app labels to get app categories here 

In [4]:
# Reading app labels
# Question: 1 device can have many events. many events can have many app id?
# Read app labels to get app categories and then groupby device id to get if there are more than app is being read
apl = pd.read_csv('app_labels.csv')
lct = pd.read_csv('label_categories.csv')

In [5]:
act = pd.merge(apl, lct, how='left', on='label_id')

In [6]:
act.head()

,app_id,label_id,category
0,7324884708820027918,251,Finance
1,-4494216993218550286,251,Finance
2,6058196446775239644,406,unknown
3,6058196446775239644,407,DS_P2P net loan
4,8694625920731541625,406,unknown


In [7]:
act.to_csv('app_categories.csv')

In [8]:
ape.drop_duplicates('event_id', keep='first', inplace=True)

In [9]:
ape = pd.merge(ape, act, how='left', on='app_id')

In [10]:
ape.head()

,event_id,app_id,is_installed,is_active,label_id,category
0,2,5927333115845830913,1,1,549,Property Industry 1.0
1,2,5927333115845830913,1,1,710,Relatives 1
2,2,5927333115845830913,1,1,704,Property Industry 2.0
3,2,5927333115845830913,1,1,548,Industry tag
4,2,5927333115845830913,1,1,172,IM


In [11]:
ape.drop(['app_id'], axis=1, inplace=True)

In [12]:
ape.head()

,event_id,is_installed,is_active,label_id,category
0,2,1,1,549,Property Industry 1.0
1,2,1,1,710,Relatives 1
2,2,1,1,704,Property Industry 2.0
3,2,1,1,548,Industry tag
4,2,1,1,172,IM


In [13]:
events = pd.read_csv("events.csv", dtype={'device_id': np.str})

In [14]:
events.head()

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


In [15]:
# The idea here is to count the number of installed apps using the data
# from app_events.csv above. Also to count the number of active apps.
events = pd.merge(events, ape, how='left', on='event_id', left_index=True)

In [16]:
events.head()

,event_id,device_id,timestamp,longitude,latitude,is_installed,is_active,label_id,category
8463466,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,NaN,NaN,NaN,NaN
0,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,1.0,1.0,549.0,Property Industry 1.0
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,1.0,1.0,710.0,Relatives 1
2,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,1.0,1.0,704.0,Property Industry 2.0
3,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,1.0,1.0,548.0,Industry tag


In [30]:
events_small = events[['device_id', 'is_installed', 'is_active', 'category']].drop_duplicates('device_id', keep='first')

In [31]:
events_small.head()

,device_id,is_installed,is_active,category
8463466,29182687948017175,NaN,NaN,NaN
0,-6401643145415154744,1.0,1.0,Property Industry 1.0
8463466,-4833982096941402721,NaN,NaN,NaN
8463466,-6815121365017318426,NaN,NaN,NaN
8463466,-5373797595892518570,NaN,NaN,NaN


In [32]:
# Let's one-hot encode category
for Category in set(events_small.category):
    #add column to events small called Category
    events_small[Category] = np.where(events_small.category == Category,1,0)

In [33]:
events_small.head()

,device_id,is_installed,is_active,category,nan,Traditional securities brokerage,reading platform,show,radio,SLG (strategy),...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
8463466,29182687948017175,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,-6401643145415154744,1.0,1.0,Property Industry 1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8463466,-4833982096941402721,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8463466,-6815121365017318426,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8463466,-5373797595892518570,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
pbd = pd.read_csv("phone_brand_device_model.csv", dtype={'device_id': np.str})

In [40]:
pbd.drop_duplicates('device_id', keep='first', inplace=True)

In [41]:
pbd.head()

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


In [42]:
english_phone_brands_mapping = {
    "三星": "samsung",
    "天语": "Ktouch",
    "海信": "hisense",
    "联想": "lenovo",
    "欧比": "obi",
    "爱派尔": "ipair",
    "努比亚": "nubia",
    "优米": "youmi",
    "朵唯": "dowe",
    "黑米": "heymi",
    "锤子": "hammer",
    "酷比魔方": "koobee",
    "美图": "meitu",
    "尼比鲁": "nibilu",
    "一加": "oneplus",
    "优购": "yougo",
    "诺基亚": "nokia",
    "糖葫芦": "candy",
    "中国移动": "ccmc",
    "语信": "yuxin",
    "基伍": "kiwu",
    "青橙": "greeno",
    "华硕": "asus",
    "夏新": "panosonic",
    "维图": "weitu",
    "艾优尼": "aiyouni",
    "摩托罗拉": "moto",
    "乡米": "xiangmi",
    "米奇": "micky",
    "大可乐": "bigcola",
    "沃普丰": "wpf",
    "神舟": "hasse",
    "摩乐": "mole",
    "飞秒": "fs",
    "米歌": "mige",
    "富可视": "fks",
    "德赛": "desci",
    "梦米": "mengmi",
    "乐视": "lshi",
    "小杨树": "smallt",
    "纽曼": "newman",
    "邦华": "banghua",
    "E派": "epai",
    "易派": "epai",
    "普耐尔": "pner",
    "欧新": "ouxin",
    "西米": "ximi",
    "海尔": "haier",
    "波导": "bodao",
    "糯米": "nuomi",
    "唯米": "weimi",
    "酷珀": "kupo",
    "谷歌": "google",
    "昂达": "ada",
    "聆韵": "lingyun",
    "小米": "Xiaomi",
    "华为": "Huawei",
    "魅族": "Meizu",
    "中兴": "ZTE",
    "酷派": "Coolpad",
    "金立": "Gionee",
    "SUGAR": "SUGAR",
    "OPPO": "OPPO",
    "vivo": "vivo",
    "HTC": "HTC",
    "LG": "LG",
    "ZUK": "ZUK",
    "TCL": "TCL",
    "LOGO": "LOGO",
    "SUGAR": "SUGAR",
    "Lovme": "Lovme",
    "PPTV": "PPTV",
    "ZOYE": "ZOYE",
    "MIL": "MIL",
    "索尼" : "Sony",
    "欧博信" : "Opssom",
    "奇酷" : "Qiku",
    "酷比" : "CUBE",
    "康佳" : "Konka",
    "亿通" : "Yitong",
    "金星数码" : "JXD",
    "至尊宝" : "Monkey King",
    "百立丰" : "Hundred Li Feng",
    "贝尔丰" : "Bifer",
    "百加" : "Bacardi",
    "诺亚信" : "Noain",
    "广信" : "Kingsun",
    "世纪天元" : "Ctyon",
    "青葱" : "Cong",
    "果米" : "Taobao",
    "斐讯" : "Phicomm",
    "长虹" : "Changhong",
    "欧奇" : "Oukimobile",
    "先锋" : "XFPLAY",
    "台电" : "Teclast",
    "大Q" : "Daq",
    "蓝魔" : "Ramos",
    "奥克斯" : "AUX"
}

In [43]:
# Now, we map the english translation into the current phone_brand
pbd.phone_brand = pbd.phone_brand.map(pd.Series(english_phone_brands_mapping), na_action='ignore')

In [44]:
pbd.head()

,device_id,phone_brand,device_model
0,-8890648629457979026,Xiaomi,红米
1,1277779817574759137,Xiaomi,MI 2
2,5137427614288105724,samsung,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,samsung,Galaxy Note 2


In [45]:
# One-hot encode phone brand here
for Brand in set(pbd.phone_brand):
    #add column to phonecases called colour
    pbd[Brand] = np.where(pbd.phone_brand == Brand,1,0)

In [46]:
pbd.head()

,device_id,phone_brand,device_model,nan,MIL,ipair,Huawei,Bifer,Qiku,mige,...,yuxin,Taobao,yougo,wpf,Oukimobile,samsung,panosonic,asus,oneplus,bodao
0,-8890648629457979026,Xiaomi,红米,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1277779817574759137,Xiaomi,MI 2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5137427614288105724,samsung,Galaxy S4,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3669464369358936369,SUGAR,时尚手机,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-5019277647504317457,samsung,Galaxy Note 2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [47]:
# One-hot encode phone brand here
for Model in set(pbd.device_model):
    #add column to phonecases called colour
    pbd[Model] = np.where(pbd.device_model == Model,1,0)

In [48]:
pbd.head()

,device_id,phone_brand,device_model,nan,MIL,ipair,Huawei,Bifer,Qiku,mige,...,GN9002,s800,M8x,D557,ZenFone 2,G309T,ZenFone 5,Xperia C3,G620-L72,G Pro Lite Dual
0,-8890648629457979026,Xiaomi,红米,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1277779817574759137,Xiaomi,MI 2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5137427614288105724,samsung,Galaxy S4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3669464369358936369,SUGAR,时尚手机,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-5019277647504317457,samsung,Galaxy Note 2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
#Train
train = pd.read_csv("gender_age_train.csv", dtype={'device_id': np.str})

In [50]:
# One-hot encode target variable
for Group in set(train.group):
    #add column to phonecases called colour
    train[Group] = np.where(train.group == Group,1,0)

In [51]:
train.head()

,device_id,gender,age,group,F27-28,M39+,F43+,M23-26,M32-38,M22-,F33-42,F23-,F24-26,F29-32,M27-28,M29-31
0,-8076087639492063270,M,35,M32-38,0,0,0,0,1,0,0,0,0,0,0,0
1,-2897161552818060146,M,35,M32-38,0,0,0,0,1,0,0,0,0,0,0,0
2,-8260683887967679142,M,35,M32-38,0,0,0,0,1,0,0,0,0,0,0,0
3,-4938849341048082022,M,30,M29-31,0,0,0,0,0,0,0,0,0,0,0,1
4,245133531816851882,M,30,M29-31,0,0,0,0,0,0,0,0,0,0,0,1


In [52]:
train = train.drop(['age'], axis=1)
train = train.drop(['gender'], axis=1)

In [53]:
train = pd.merge(train, pbd, how='left', on='device_id', left_index=True)

In [54]:
train.head()

,device_id,group,F27-28,M39+,F43+,M23-26,M32-38,M22-,F33-42,F23-,...,GN9002,s800,M8x,D557,ZenFone 2,G309T,ZenFone 5,Xperia C3,G620-L72,G Pro Lite Dual
56800,-8076087639492063270,M32-38,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41294,-2897161552818060146,M32-38,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9993,-8260683887967679142,M32-38,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179893,-4938849341048082022,M29-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102570,245133531816851882,M29-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
train = pd.merge(train, events_small, how='left', on='device_id', left_index=True)

In [56]:
train.head()

,device_id,group,F27-28,M39+,F43+,M23-26,M32-38,M22-,F33-42,F23-,...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
8460527,-8076087639492063270,M32-38,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8460527,-2897161552818060146,M32-38,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6455195,-8260683887967679142,M32-38,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8460527,-4938849341048082022,M29-31,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8460527,245133531816851882,M29-31,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Impute the N/A with -1
train.fillna(-1, inplace=True)

In [58]:
train.head()

,device_id,group,F27-28,M39+,F43+,M23-26,M32-38,M22-,F33-42,F23-,...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
8460527,-8076087639492063270,M32-38,0,0,0,0,1,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8460527,-2897161552818060146,M32-38,0,0,0,0,1,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6455195,-8260683887967679142,M32-38,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8460527,-4938849341048082022,M29-31,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8460527,245133531816851882,M29-31,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [59]:
train.shape[0]

74645

In [60]:
# Test
test = pd.read_csv("gender_age_test.csv", dtype={'device_id': np.str})

In [61]:
test.head()

,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277
3,-6220210354783429585
4,-5893464122623104785


In [62]:
test = pd.merge(test, pbd, how='left', on='device_id', left_index=True)

In [63]:
test.head()

,device_id,phone_brand,device_model,nan,MIL,ipair,Huawei,Bifer,Qiku,mige,...,GN9002,s800,M8x,D557,ZenFone 2,G309T,ZenFone 5,Xperia C3,G620-L72,G Pro Lite Dual
78253,1002079943728939269,Xiaomi,小米note,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84501,-1547860181818787117,Xiaomi,红米2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104765,7374582448058474277,Huawei,Y523-L176,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
145592,-6220210354783429585,Huawei,荣耀6,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12144,-5893464122623104785,Xiaomi,MI 2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
test = pd.merge(test, events_small, how='left', on='device_id', left_index=True)

In [65]:
test.head()

,device_id,phone_brand,device_model,nan_x,MIL,ipair,Huawei,Bifer,Qiku,mige,...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
1199784,1002079943728939269,Xiaomi,小米note,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
482597,-1547860181818787117,Xiaomi,红米2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243286,7374582448058474277,Huawei,Y523-L176,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
844354,-6220210354783429585,Huawei,荣耀6,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8460527,-5893464122623104785,Xiaomi,MI 2,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
test.fillna(-1, inplace=True)

In [67]:
test.head()

,device_id,phone_brand,device_model,nan_x,MIL,ipair,Huawei,Bifer,Qiku,mige,...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
1199784,1002079943728939269,Xiaomi,小米note,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
482597,-1547860181818787117,Xiaomi,红米2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243286,7374582448058474277,Huawei,Y523-L176,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
844354,-6220210354783429585,Huawei,荣耀6,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8460527,-5893464122623104785,Xiaomi,MI 2,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [68]:
test.shape[0]

112071

In [69]:
features = list(test.columns.values)

In [70]:
features

['device_id',
 'phone_brand',
 'device_model',
 'nan_x',
 'MIL',
 'ipair',
 'Huawei',
 'Bifer',
 'Qiku',
 'mige',
 'Daq',
 'ada',
 'Teclast',
 'OPPO',
 'lingyun',
 'nokia',
 'Yitong',
 'newman',
 'lenovo',
 'Lovme',
 'LOGO',
 'weitu',
 'mengmi',
 'Ktouch',
 'weimi',
 'desci',
 'aiyouni',
 'greeno',
 'google',
 'Konka',
 'CUBE',
 'Meizu',
 'epai',
 'nibilu',
 'hammer',
 'youmi',
 'Ctyon',
 'TCL',
 'xiangmi',
 'nubia',
 'Sony',
 'Opssom',
 'smallt',
 'banghua',
 'SUGAR',
 'Hundred Li Feng',
 'candy',
 'HTC',
 'obi',
 'Bacardi',
 'Cong',
 'ccmc',
 'JXD',
 'Coolpad',
 'lshi',
 'ZUK',
 'dowe',
 'XFPLAY',
 'Xiaomi',
 'hasse',
 'hisense',
 'meitu',
 'Phicomm',
 'ZTE',
 'ZOYE',
 'AUX',
 'kiwu',
 'Monkey King',
 'ximi',
 'Gionee',
 'vivo',
 'kupo',
 'haier',
 'Noain',
 'bigcola',
 'micky',
 'moto',
 'Changhong',
 'heymi',
 'PPTV',
 'mole',
 'ouxin',
 'LG',
 'koobee',
 'Kingsun',
 'pner',
 'Ramos',
 'fks',
 'nuomi',
 'fs',
 'yuxin',
 'Taobao',
 'yougo',
 'wpf',
 'Oukimobile',
 'samsung',
 'panos

In [71]:
features.remove('device_id')

In [72]:
features

['phone_brand',
 'device_model',
 'nan_x',
 'MIL',
 'ipair',
 'Huawei',
 'Bifer',
 'Qiku',
 'mige',
 'Daq',
 'ada',
 'Teclast',
 'OPPO',
 'lingyun',
 'nokia',
 'Yitong',
 'newman',
 'lenovo',
 'Lovme',
 'LOGO',
 'weitu',
 'mengmi',
 'Ktouch',
 'weimi',
 'desci',
 'aiyouni',
 'greeno',
 'google',
 'Konka',
 'CUBE',
 'Meizu',
 'epai',
 'nibilu',
 'hammer',
 'youmi',
 'Ctyon',
 'TCL',
 'xiangmi',
 'nubia',
 'Sony',
 'Opssom',
 'smallt',
 'banghua',
 'SUGAR',
 'Hundred Li Feng',
 'candy',
 'HTC',
 'obi',
 'Bacardi',
 'Cong',
 'ccmc',
 'JXD',
 'Coolpad',
 'lshi',
 'ZUK',
 'dowe',
 'XFPLAY',
 'Xiaomi',
 'hasse',
 'hisense',
 'meitu',
 'Phicomm',
 'ZTE',
 'ZOYE',
 'AUX',
 'kiwu',
 'Monkey King',
 'ximi',
 'Gionee',
 'vivo',
 'kupo',
 'haier',
 'Noain',
 'bigcola',
 'micky',
 'moto',
 'Changhong',
 'heymi',
 'PPTV',
 'mole',
 'ouxin',
 'LG',
 'koobee',
 'Kingsun',
 'pner',
 'Ramos',
 'fks',
 'nuomi',
 'fs',
 'yuxin',
 'Taobao',
 'yougo',
 'wpf',
 'Oukimobile',
 'samsung',
 'panosonic',
 'asus'

# All pre-processing are done and ready to play with modelling here!

# Logistic Regression

In [73]:
train.to_csv('train_one_hot.csv')

In [74]:
test.to_csv('test_one_hot.csv')

In [75]:
import sklearn.linear_model

In [76]:
regressor = sklearn.linear_model.LogisticRegression()

In [87]:
X = train.drop(['device_id', 'phone_brand', 'device_model', 'group', 'F27-28', 'M39+', 'F43+', 'M23-26', 'M32-38',
               'M22-', 'F33-42', 'F24-26', 'F29-32', 'M27-28', 'F23-', 'M29-31'], axis=1)

In [88]:
X.head()

,nan_x,MIL,ipair,Huawei,Bifer,Qiku,mige,Daq,ada,Teclast,...,Internet Securities,blogs,Personal Effectiveness 1,Hotel Chain,Weather,Financial Services,foreign language,Accounting,"online shopping by group, like groupon",casual games
8460527,0,0,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8460527,0,0,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6455195,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8460527,0,0,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8460527,0,0,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [89]:
y = train[['F27-28', 'M39+', 'F43+', 'M23-26', 'M32-38',
               'M22-', 'F33-42', 'F24-26', 'F29-32', 'M27-28', 'F23-', 'M29-31']]

In [90]:
y.head()

,F27-28,M39+,F43+,M23-26,M32-38,M22-,F33-42,F24-26,F29-32,M27-28,F23-,M29-31
8460527,0,0,0,0,1,0,0,0,0,0,0,0
8460527,0,0,0,0,1,0,0,0,0,0,0,0
6455195,0,0,0,0,1,0,0,0,0,0,0,0
8460527,0,0,0,0,0,0,0,0,0,0,0,1
8460527,0,0,0,0,0,0,0,0,0,0,0,1


In [91]:
regressor.fit(X, y)

ValueError: could not convert string to float: Property Industry 1.0

array([False, False, False, ..., False, False, False], dtype=bool)